In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

import tensorflow as tf
from tensorflow import keras
import os
import datetime

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [9]:
# 리눅스 한글 설정
plt.rc('font', family = 'NanumGothic')

In [10]:
file = '2013~2023 한국 기온,습도,풍속.csv'
file2 = '한국기후 평균 최고 최저.csv'

In [11]:
df = pd.read_csv(file, encoding = 'cp949')
df1 = pd.read_csv(file2, encoding = 'cp949', header = 0)
df1.head()

/tmp/ipykernel_2761/207623365.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(file2, encoding = 'cp949', header = 0)


,\t\t지점번호,지점명,일시,평균기온(℃),최고기온(℃),최저기온(℃)
0,105,강릉,1970-01-01,3.2,10.3,-0.5
1,105,강릉,1970-01-02,1.8,5.1,-0.2
2,105,강릉,1970-01-03,-0.3,4.8,-2.2
3,105,강릉,1970-01-04,-5.9,-1.7,-12.6
4,105,강릉,1970-01-05,-7.9,-0.8,-14.7


In [12]:
df1.columns = ['지점번호', '지점명', '일시', '평균기온(℃)', '최고기온(℃)', '최저기온(℃)']
df1 = df1.iloc[:, 2:]
df1.head()

,일시,평균기온(℃),최고기온(℃),최저기온(℃)
0,1970-01-01,3.2,10.3,-0.5
1,1970-01-02,1.8,5.1,-0.2
2,1970-01-03,-0.3,4.8,-2.2
3,1970-01-04,-5.9,-1.7,-12.6
4,1970-01-05,-7.9,-0.8,-14.7


In [13]:
df_korea = df.drop(['지점번호', '지점명'], axis = 1).groupby(['일시']).mean()
df_temp = df1.drop(['최저기온(℃)'], axis = 1).groupby(['일시']).mean()
df_korea.head(10)

,평균기온(℃),최고기온(℃),최저기온(℃),평균습도(%rh),평균풍속(m/s)
일시,,,,,
2013-01-01,-0.76250,4.43750,-6.21250,73.10000,2.33125
2013-01-02,-4.70000,0.54375,-8.98125,53.29375,3.58750
2013-01-03,-8.09375,-3.99375,-11.26250,48.90000,3.40625
2013-01-04,-6.74375,-1.15000,-11.61875,50.25000,2.03125
2013-01-05,-3.81250,1.75625,-9.41875,57.06250,2.28125
2013-01-06,-2.32500,2.52500,-5.73750,57.04375,1.96875
2013-01-07,-2.00625,3.35000,-6.72500,49.62500,1.95000
2013-01-08,-1.22500,3.76875,-5.28750,50.66250,2.15625
2013-01-09,-4.01875,-0.80625,-7.14375,45.38125,3.41250


In [14]:
filter1 = df_temp['최고기온(℃)'].isna() == True
df_temp['최고기온(℃)'][filter1] = 20.7875

In [15]:
df_korea = df_korea.reset_index()

In [16]:
df_korea['일시']  = pd.to_datetime(df_korea['일시'])
df_korea.set_index('일시', inplace = True)

In [17]:
dataset = df_korea.iloc[:, :3]
#dataset = df_temp.iloc[:, :]
dataset = pd.DataFrame(dataset)
data = dataset.values

data.shape

(3879, 3)

In [18]:
train_size = int(len(data)*0.75)
test_size = len(data) - train_size
print('Train Size :', train_size, 'Test Size : ', test_size)

Train Size : 2909 Test Size :  970


In [19]:
scaler = MinMaxScaler(feature_range = (0 , 1))
scaled_data = scaler.fit_transform(np.array(data))

In [20]:
train_data = scaled_data[0:train_size, :]
train_data.shape

(2909, 3)

In [21]:
x_train = []
y_train = []
time_step = 60
n_col = 3

for i in range(time_step, len(scaled_data)):
    print(time_step, i)
    x_train.append(scaled_data[i-time_step:i, :n_col])
    y_train.append(scaled_data[i, :n_col])
    if i <= time_step:
        print('x_train : ', x_train)
        print('y_train : ', y_train)

60 60
x_train :  [array([[0.23578342, 0.26223264, 0.18837736],
       [0.14050212, 0.17096396, 0.12150943],
       [0.05837871, 0.06460592, 0.06641509],
       [0.09104658, 0.13126282, 0.05781132],
       [0.16197822, 0.19938471, 0.1109434 ],
       [0.19797338, 0.21740404, 0.19984906],
       [0.20568663, 0.23674187, 0.176     ],
       [0.22459165, 0.24655728, 0.21071698],
       [0.1569873 , 0.13932025, 0.16588679],
       [0.138536  , 0.16158805, 0.12316981],
       [0.20689655, 0.24846176, 0.1450566 ],
       [0.31049607, 0.34178142, 0.28256604],
       [0.29567453, 0.27629651, 0.296     ],
       [0.288415  , 0.30969821, 0.27577358],
       [0.26225045, 0.28581893, 0.23833962],
       [0.27253479, 0.26530911, 0.27456604],
       [0.20054446, 0.17594492, 0.21841509],
       [0.21264368, 0.24948725, 0.16935849],
       [0.29007864, 0.31365368, 0.27592453],
       [0.33378705, 0.33753296, 0.2914717 ],
       [0.37658802, 0.31306768, 0.4285283 ],
       [0.36191773, 0.30105479, 0.416

In [22]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [23]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], n_col))

In [24]:
x_train.shape, y_train.shape

((3819, 60, 3), (3819, 3))

In [25]:
m_lstm = keras.Sequential([
    keras.layers.LSTM(50, input_shape=(x_train.shape[1], n_col),
                       activation = 'relu',return_sequences = True),
    keras.layers.LSTM(64, return_sequences = True, activation = 'relu'),
    keras.layers.LSTM(50, return_sequences = True, activation = 'relu'),
    keras.layers.LSTM(64, return_sequences = False, activation = 'relu'), 
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(16, activation = 'relu'),
    keras.layers.Dense(n_col)
])
m_lstm.summary()
#kernel_regularizer="l2"

2023-08-25 15:14:39.011442: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 50)            10800     
                                                                 
 lstm_1 (LSTM)               (None, 60, 64)            29440     
                                                                 
 lstm_2 (LSTM)               (None, 60, 50)            23000     
                                                                 
 lstm_3 (LSTM)               (None, 64)                29440     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 3)                 5

In [26]:
m_lstm.compile(optimizer = tf.optimizers.RMSprop(learning_rate = 0.01),
             loss = keras.losses.MeanSquaredError(),
             metrics = ['mae'])

In [27]:
h = m_lstm.fit(x_train, y_train, epochs = 50, shuffle=False, batch_size = 32)

Epoch 1/50
120/120 [==============================] - 7s 39ms/step - loss: 12235219.0000 - mae: 259.3683
Epoch 2/50
120/120 [==============================] - 5s 39ms/step - loss: 0.0472 - mae: 0.1862
Epoch 3/50
120/120 [==============================] - 5s 39ms/step - loss: 0.0347 - mae: 0.1598
Epoch 4/50
120/120 [==============================] - 5s 39ms/step - loss: 0.0352 - mae: 0.1631
Epoch 5/50
120/120 [==============================] - 5s 39ms/step - loss: 0.0272 - mae: 0.1257
Epoch 6/50
120/120 [==============================] - 5s 39ms/step - loss: 0.0173 - mae: 0.1087
Epoch 7/50
120/120 [==============================] - 5s 39ms/step - loss: 0.0152 - mae: 0.0993
Epoch 8/50
120/120 [==============================] - 5s 39ms/step - loss: 1290972928.0000 - mae: 2311.2229
Epoch 9/50
120/120 [==============================] - 5s 39ms/step - loss: 0.0072 - mae: 0.0649
Epoch 10/50
120/120 [==============================] - 5s 39ms/step - loss: 10.5048 - mae: 0.2325
Epoch 11/50
120/1

In [ ]:
m_lstm.save('m4_lstm_RMSprop.h5')

In [ ]:
m_lstm_RMSprop = tf.keras.models.load_model('m4_lstm_RMSprop.h5')

In [ ]:
test_data = scaled_data[train_size - time_step:, :]

x_test = []
y_test = []

for i in range(time_step, len(test_data)):
    #print(time_step, i)
    x_test.append(test_data[i-time_step:i, 0:n_col])
    y_test.append(test_data[i, 0:n_col])
    
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], n_col))

x_test

In [ ]:
m_lstm_RMSprop.evaluate(x_test, y_test)

In [ ]:
pred = m_lstm_RMSprop.predict(x_test)
pred

In [ ]:
pred = scaler.inverse_transform(pred)
print(pred.shape)
pred

In [ ]:
y_test = scaler.inverse_transform(y_test)
y_test

In [ ]:
pred = pd.DataFrame(pred)
y_test = pd.DataFrame(y_test)

In [ ]:
pred.columns = ['예측평균기온', '예측최고기온', '예측최저기온']
y_test.columns = ['실제평균기온', '실제최고기온', '실제최저기온']

#pred.columns = ['예측평균기온', '예측최고기온']
#y_test.columns = ['실제평균기온', '실제최고기온']

In [ ]:
df_pred = pd.concat((pred, y_test), axis = 1)
df_pred = df_pred[['예측평균기온', '실제평균기온', '예측최고기온', '실제최고기온', '예측최저기온', '실제최저기온']]
#df_pred = df_pred[['예측평균기온', '실제평균기온', '예측최고기온', '실제최고기온']]
df_pred

In [ ]:
fig, axe = plt.subplots(2,1, figsize = (16, 16))
axe[0].plot(df_pred['실제평균기온'])
axe[1].plot(df_pred['예측평균기온'], c = 'g')

axe[0].set_title('실제 평균 기온', size = 20, weight = 'bold')
axe[1].set_title('예측 평균 기온', size = 20, weight = 'bold')
#axe.legend(['actual', 'predict'])

In [ ]:
fig, axe = plt.subplots(2,1, figsize = (16, 16))
axe[0].plot(df_pred['실제최고기온'])
axe[1].plot(df_pred['예측최고기온'], c = 'r')

axe[0].set_title('실제 최고 기온', size = 20, weight = 'bold')
axe[1].set_title('예측 최고 기온', size = 20, weight = 'bold')

#axe.legend(['actual', 'predict'])

In [ ]:

fig, axe = plt.subplots(2,1, figsize = (16, 16))
axe[0].plot(df_pred['실제최저기온'])
axe[1].plot(df_pred['예측최저기온'], c = 'b')

axe[0].set_title('실제 최저 기온', size = 20, weight = 'bold')
axe[1].set_title('예측 최저 기온', size = 20, weight = 'bold')


In [ ]:
start_date = datetime.date(2023, 8, 16)
#print('언제의 대한민국 평균 기온을 알고 싶으신가요? (Ex. 2040-01-01, 형태로 적어주세요)')
year, month, day = map(int, input().split('-'))
target_date = datetime.date(year, month, day)

d_day = (target_date - start_date).days
print(d_day)

In [ ]:
list(x_test[-1][0])

In [ ]:
last_array = []
for i in range(len(x_test[-1])):
    last_array.append(list(x_test[-1][i]))
len(last_array)
np.array(last_array)

In [ ]:
for i in range(time_step, d_day +time_step):
    print(time_step, i)
    last_ar = np.array(last_array)[i-time_step:i, 0:n_col]
    print(last_ar)
    array = np.reshape(last_ar, (1, last_ar.shape[0], n_col))
    print(array.shape)
    last_array.append(list(m_lstm_RMSprop.predict(array)[0]))
    print(len(last_array))

In [ ]:
test_pred = np.array(last_array).reshape(len(last_array), n_col)

In [ ]:
test_pred = scaler.inverse_transform(test_pred)
test_pred = pd.DataFrame(test_pred)
#test_pred.columns = ['미래예측평균기온', '미래예측최고기온', '미래예측최저기온']
test_pred

In [ ]:
fig, axe = plt.subplots(figsize = (16, 6))
axe.plot(test_pred)